In [13]:
# To Save back
# http://www.countingcalculi.com/explanations/google_sheets_and_jupyter_notebooks/
# from df2gspread import df2gspread as d2g
# wks_name = 'Jupyter Manipulated Data'
# d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

ModuleNotFoundError: No module named 'df2gspread'

In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
# Authorise with Google Docs
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/data/docs/JupyterSheets.json', scope)
gc = gspread.authorize(credentials)

In [48]:
# Load Spreadsheet
spreadsheet_key = '1KQwNxNVYJ21K1LrDcAddEj1_yxSWlxVpc6xS9z542yY'
book = gc.open_by_key(spreadsheet_key)
UK = pd.DataFrame(book.worksheet("UKTxns").get_all_records())
US = pd.DataFrame(book.worksheet("USTxns").get_all_records())

In [35]:
def sort_tables(df):
    cols = ['Account', 'Date', 'Payee', 'Type', 'Category', 'Memo', 'Outflow', 'Inflow', 'Net']
    df['Date'] = pd.to_datetime(df['Date'])
    df['Inflow'] = pd.to_numeric(df['Inflow'])
    df['Outflow'] = pd.to_numeric(df['Outflow'])
    df['Inflow'].fillna(value=0, inplace=True)
    df['Outflow'].fillna(value=0, inplace=True)
    df['Net'] = df['Inflow'] - df ['Outflow']
    #df['Net'] = pd.to_numeric(df['Net'])
    df = df[cols]
    return df

In [58]:
def add_date_info(df):
    """
    Adds columns for year, month, quarter, week number
    """
    df['Year'], df['Month'] = df['Date'].dt.year, df['Date'].dt.month
    df['Quarter'], df['Week'] = df['Date'].dt.quarter, df['Date'].dt.week
    #df['Year'] = df['Year'].astype('int')
    #df['Month'] = df['Month'].astype('int')
    #df['Week'] = df['Week'].astype('int')
    #df['Quarter'] = df['Quarter'].astype('int')

In [59]:
sort_tables(UK)
add_date_info(UK)
#cols = ['Account', 'Date', 'Payee', 'Type', 'Category', 'Memo', 'Outflow', 'Inflow', 'Net']
#UK = UK[cols]

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [50]:
sort_tables(US)
add_date_info(US)


In [51]:
US.head()

,Account,Category,Date,Inflow,Memo,Outflow,Payee,Type,Net,Year,Month,Quarter,Week
0,US HSBC Checking,,2015-01-01,2585.55,,0.00,Starting Balance,Transfer,2585.55,2015,1,1,1
1,US HSBC World,,2015-01-01,0.00,,3284.48,Starting Balance,Transfer,-3284.48,2015,1,1,1
2,US HSBC World,Zisc,2015-01-02,0.00,Returning my Tshirt,3.22,USPS,Expense,-3.22,2015,1,1,1
3,US HSBC World,Food,2015-01-02,0.00,,6.50,KR Cafe,Expense,-6.50,2015,1,1,1
4,US HSBC World,Entertainment,2015-01-02,0.00,,17.82,SiriusXM,Expense,-17.82,2015,1,1,1


In [57]:
UK.dtypes

Account             object
Category            object
Date        datetime64[ns]
Inflow             float64
Memo                object
Outflow            float64
Payee               object
Type                object
Net                float64
Year               float64
Month              float64
Quarter            float64
Week               float64
dtype: object